# First Keras project

Keeping it simple. Probably a little too simple, but let's finish this, learn what we learn and move on to something more educational.

(I'm working with Kaggle's MNIST digist data. It's all preprocessed except for one little thing, which is why this is too easy.)

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

keras.__version__

'2.3.0-tf'

In [2]:
#tf.python.client.device_lib.list_local_devices() #verifies GPU type.
tf.config.experimental.list_physical_devices('GPU')#verifies GPU detected

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
dataframe = pd.read_csv('data/train.csv')
dev_df = dataframe.sample(n=3000, random_state=1)
train_df = dataframe.drop(dev_df.index)
assert train_df.shape[1] == 785 #should be 784 + 1

In [5]:
def dataframe_to_dataset(dataframe, batch_size=64, label='label'):
    ds = tf.data.Dataset.from_tensor_slices((dataframe.drop(label,axis=1).to_numpy(), dataframe[label]))
    ds.shuffle(buffer_size=len(dataframe))
    ds.batch(batch_size)
    return ds

dev_ds = dataframe_to_dataset(dev_df)
train_ds = dataframe_to_dataset(train_df)

In [6]:
from tensorflow.keras import layers

##Build the model
inputs = keras.Input(shape=(784))
x = layers.experimental.preprocessing.Rescaling(1./255)(inputs)
x = layers.Dense(100, activation='relu')(x)
x = layers.Dense(100, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)
model = keras.Model(inputs=inputs,outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
rescaling (Rescaling)        (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               78500     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 89,610
Trainable params: 89,610
Non-trainable params: 0
_________________________________________________________________


In [12]:
## Cool. 
## Compile and train

model.compile(optimizer="adam", loss="categorical_crossentropy")
(X, y) = (train_df.drop('label',axis=1).to_numpy(), train_df['label'].to_numpy())
print(y.shape)
assert X.shape[1] == 784
assert X.shape[0] == y.shape[0]
history = model.fit(X, y, epochs=1, batch_size=64)
#history = model.fit(train_ds,epochs=1)
#history

(39000,)


ValueError: in user code:

    /home/lenhart/anaconda3/envs/tf-gpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/lenhart/anaconda3/envs/tf-gpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/lenhart/anaconda3/envs/tf-gpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/lenhart/anaconda3/envs/tf-gpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/lenhart/anaconda3/envs/tf-gpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:532 train_step  **
        loss = self.compiled_loss(
    /home/lenhart/anaconda3/envs/tf-g